Importar bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree, cophenet
from scipy.stats import zscore
from scipy.linalg import eigh
from scipy import linalg

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

Funções para importação do base de dados e seleção as variáveis

In [ ]:
PATH_TO_ENEM_CSV = "PATH_TO_ENEM_2023.csv"  # <- coloque o CSV aqui

def load_and_preview(path):
    df = pd.read_csv(path)
    print("Dimensões:", df.shape)
    print("Colunas:", df.columns.tolist())
    display(df.head())
    return df

def select_variables(df:pd.DataFrame, vars_list:list[str]):
    """
    Escolha as variáveis que farão parte da 'variável estatística de agrupamento'.
    (Conforme o material: selecionar variáveis relevantes; a análise é sensível a isso). :contentReference[oaicite:17]{index=17}
    """
    return df[vars_list].copy()

Detecção de outliers multivariados (Mahalanobis)

In [ ]:
def mahalanobis_distance(X):
    """
    Calcula D^2 de Mahalanobis para cada observação.
    Usado nos materiais para identificar potenciais outliers multivariados. :contentReference[oaicite:18]{index=18}
    """
    X = np.asarray(X)
    mu = X.mean(axis=0)
    cov = np.cov(X, rowvar=False)
    cov_inv = linalg.pinv(cov)
    dif = X - mu
    left = np.dot(dif, cov_inv)
    md2 = np.einsum('ij,ij->i', left, dif)
    return md2

def flag_outliers_md2(X, alpha=0.001):
    """
    alpha: nível para marcar outliers (usando quão grande é D2).
    Podemos usar um critério empírico (p-quantil) ou comparação com qui²(p).
    Aqui retornamos os índices ordenados por D2 decrescente.
    """
    md2 = mahalanobis_distance(X)
    df_indices = np.argsort(md2)[::-1]
    return md2, df_indices

Padronização

In [ ]:
def standardize_df(df:pd.DataFrame, method="zscore"):
    """
    O material recomenda z-score quando há diferentes dispersões. :contentReference[oaicite:19]{index=19}
    """
    if method == "zscore":
        scaler = StandardScaler()
        arr = scaler.fit_transform(df.values)
        df_s = pd.DataFrame(arr, index=df.index, columns=df.columns)
        return df_s, scaler
    else:
        raise ValueError("Método de padronização não implementado")

Análise hierárquica (dendrograma) + cophenetic

In [ ]:
def hierarchical_analysis(X, method="ward", metric="euclidean", plot=True):
    d = pdist(X, metric=metric)
    Z = linkage(d, method=method)
    c, coph_dists = cophenet(Z, d)  # coeficiente cofenético para avaliar dendrograma.
    if plot:
        plt.figure(figsize=(10, 5))
        dendrogram(Z, truncate_mode='level', p=30)
        plt.title(f"Dendrograma (method={method}) — coef cofenético={c:.3f}")
        plt.xlabel("Observações")
        plt.ylabel("Distância")
        plt.show()
    return Z, c

Extrair sementes a partir do corte do dendrograma

In [ ]:
def seeds_from_hierarchical(Z, X, k):
    """
    Corta o dendrograma em k grupos e retorna os centróides como seeds para kmeans.
    Recomendado combinar hierárquico -> k-means (slides). :contentReference[oaicite:22]{index=22}
    """
    labels = cut_tree(Z, n_clusters=k).reshape(-1)
    seeds = []
    for lab in np.unique(labels):
        seeds.append(X[labels == lab].mean(axis=0))
    seeds = np.vstack(seeds)
    return seeds, labels

# ==========================
# 8) K-means (com avaliação em K)
# ==========================
def kmeans_range(X, ks=range(2,8), seeds=None, n_init=10, random_state=42):
    """
    Testa vários k, retorna: inertia, silhouette para cada k.
    (Material recomenda testar intervalo de k e avaliar). :contentReference[oaicite:23]{index=23}
    """
    results = []
    for k in ks:
        if seeds is not None and seeds.shape[0] == k:
            init = seeds
            km = KMeans(n_clusters=k, init=init, n_init=1, random_state=random_state, max_iter=300)
        else:
            km = KMeans(n_clusters=k, n_init=n_init, random_state=random_state, max_iter=300)
        labels = km.fit_predict(X)
        inertia = km.inertia_
        sil = silhouette_score(X, labels) if k > 1 else np.nan
        results.append({'k': k, 'inertia': inertia, 'silhouette': sil, 'model': km, 'labels': labels})
    return results

def plot_k_diagnostics(results):
    ks = [r['k'] for r in results]
    inertias = [r['inertia'] for r in results]
    sils = [r['silhouette'] for r in results]
    fig, ax1 = plt.subplots(figsize=(9,4))
    ax1.plot(ks, inertias, '-o', label='Inertia (WGSS)')
    ax1.set_xlabel("k")
    ax1.set_ylabel("Inertia")
    ax2 = ax1.twinx()
    ax2.plot(ks, sils, '-s', color='tab:orange', label='Silhouette')
    ax2.set_ylabel("Silhouette score")
    ax1.set_xticks(ks)
    ax1.grid(True)
    ax1.set_title("Elbow (inertia) e Silhouette para seleção de k")
    fig.tight_layout()
    plt.show()

# ==========================
# 9) Perfil e interpretação
# ==========================
def cluster_profile(original_df:pd.DataFrame, cluster_labels):
    """
    Para cada cluster: contar tamanho, média das variáveis originais, e ANOVA simples (ou descriptivo).
    Material recomenda descrever centróides e interpretar características. :contentReference[oaicite:24]{index=24}
    """
    df = original_df.copy()
    df['cluster'] = cluster_labels
    profile = df.groupby('cluster').agg(['count','mean','std'])
    return profile

def plot_cluster_profiles(df_vars, labels):
    """
    Gráfico de perfis: médias padronizadas por cluster (útil para visualização). :contentReference[oaicite:25]{index=25}
    """
    df_vars = pd.DataFrame(df_vars)
    df_vars['cluster'] = labels
    centroids = df_vars.groupby('cluster').mean()
    centroids.T.plot(kind='line', marker='o', figsize=(10,5))
    plt.title("Perfil dos clusters (médias das variáveis padronizadas)")
    plt.xlabel("Variáveis")
    plt.ylabel("Média (padronizada)")
    plt.grid(True)
    plt.show()

# ==========================
# 10) Exemplo de execução sequencial
# ==========================
def run_full_pipeline(path, variables, hier_method="ward", k_candidates=range(2,8)):
    # 1) carregar
    df_all = load_and_preview(path)
    # 2) selecionar
    df_vars = select_variables(df_all, variables)
    # 4) outliers (Mahalanobis)
    md2, idx_sorted = flag_outliers_md2(df_vars.values)
    df_vars['MD2'] = md2
    # mostra top 10 MD2 (potenciais outliers)
    print("Top 10 potenciais outliers (Mahalanobis D^2):")
    display(df_vars.sort_values('MD2', ascending=False).head(10))
    # nota: decidir remover ou não conforme contexto (material descreve que outliers podem formar clusters isolados). :contentReference[oaicite:26]{index=26}
    # 5) padronizar
    X_std, scaler = standardize_df(df_vars.drop(columns=['MD2']))
    # 6) hierarquico
    Z, coph = hierarchical_analysis(X_std.values, method=hier_method, plot=True)
    print("Coeficiente cofenético:", coph, "(valores próximos a 0.8 são bons segundo material). :contentReference[oaicite:27]{index=27}")
    # 7) extrair seeds a partir de corte (escolha inicial k minimal, aqui pegamos a mediana de candidatos)
    k0 = int(np.median(list(k_candidates)))
    seeds, h_labels = seeds_from_hierarchical(Z, X_std.values, k=k0)
    print(f"Usando k0={k0} clusters para gerar seeds iniciais do hierárquico.")
    # 8) kmeans range (testar com e sem seeds)
    results_no_seeds = kmeans_range(X_std.values, ks=k_candidates)
    results_with_seeds = None
    if seeds.shape[0] == k0:
        # apenas ilustração: testamos seeds para k0
        results_with_seeds = kmeans_range(X_std.values, ks=[k0], seeds=seeds)
    # 9) plot diagnósticos
    plot_k_diagnostics(results_no_seeds)
    # 10) escolher melhor k (ex.: maior silhouette ou elbow)
    best = max(results_no_seeds, key=lambda r: r['silhouette'] if not np.isnan(r['silhouette']) else -999)
    print("Melhor k por silhouette:", best['k'], "silhouette:", best['silhouette'])
    # 11) perfil e plots
    best_model = best['model']
    labels = best['labels']
    profile = cluster_profile(df_vars.drop(columns=['MD2']), labels)
    display(profile)
    plot_cluster_profiles(X_std, labels)
    # PCA 2D plot com clusters
    pca = PCA(n_components=2)
    proj = pca.fit_transform(X_std)
    plt.figure(figsize=(7,6))
    sns.scatterplot(x=proj[:,0], y=proj[:,1], hue=labels, palette='tab10', s=40)
    plt.title(f"PCA 2D com clusters (k={best['k']})")
    plt.show()
    return {
        'df_vars': df_vars,
        'scaler': scaler,
        'hier_linkage': Z,
        'cofenetic': coph,
        'kmeans_results': results_no_seeds,
        'best': best
    }

# ==========================
# 11) Uso: ajustar variáveis conforme seu dataset ENEM
# ==========================
if __name__ == "__main__":
    # exemplo: variáveis de nota (ajuste para o nome real no CSV)
    vars_to_use = ['nota_mat', 'nota_por', 'nota_nat', 'nota_hum', 'nota_redacao']
    # Rode o pipeline (trocando PATH_TO_ENEM_CSV no topo)
    out = run_full_pipeline(PATH_TO_ENEM_CSV, vars_to_use, missing_strategy="drop", hier_method="ward", k_candidates=range(2,9))
    # Salve labels no arquivo original (exemplo)
    best_labels = out['best']['labels']
    df_in = pd.read_csv(PATH_TO_ENEM_CSV)
    df_in = df_in.loc[out['df_clean'].index]  # manter mesma ordem/índices limpos
    df_in['cluster'] = best_labels
    df_in.to_csv("enem_2023_clusters.csv", index=False)
    print("Clusters salvos em enem_2023_clusters.csv")